# Домашняя работа 4
### Зенина Алёна БКЛ-201

In [2]:
import numpy as np
import pandas as pd

#### 1. Реализация RSA

In [3]:
class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior)
    
    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        sp = self.speaker().T
        return rownorm(sp * self.prior)

def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T

def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)

def display_reference_game(mod):
    d = mod.lexicon.copy()
    d['costs'] = mod.costs
    d.loc['prior'] = list(mod.prior) + [""]
    d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
    display(d)

#### 2. Референциальная игра

In [6]:
msgs = ['шляпа', 'очки', 'усы']
states = ['r1', 'r2', 'r3']
lexicon1 = pd.DataFrame([[1, 1, 0],
                   [0, 0, 1],
                   [0, 1, 0]], index=msgs, columns=states)
prior = [1/3, 1/3, 1/3]
costs = [0, 0, 0]
basic_mod = RSA(lexicon=lexicon1, prior=prior, costs=costs, alpha=1.0)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.000000,1.0,0.0,0
очки,0.000000,0.0,1.0,0
усы,0.000000,1.0,0.0,0
prior,0.333333,0.333333,0.333333,
alpha,1.000000,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.75,0.25,0.0
очки,0.00,0.00,1.0
усы,0.00,1.00,0.0


Да, скалярная импликатура выводится, поскольку "очки" и "усы" более информативны: сообщение "усы" описывает только ситуацию r2, сообщение "очки" -- r3, а "шляпа" -- и r1, и r2. Если "очки" -> r3, "усы" -> r2, "шляпа" -> r1

In [7]:
msgs = ['шляпа', 'очки', 'усы']
states = ['r1', 'r2', 'r3']
lexicon2 = pd.DataFrame([[1, 1, 0],
                   [0, 0, 1],
                   [0, 1, 0]], index=msgs, columns=states)
prior = [0.1, 0.45, 0.45]
costs = [0, 0, 0]
basic_mod = RSA(lexicon=lexicon2, prior=prior, costs=costs, alpha=1.0)
display_reference_game(basic_mod)
print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
шляпа,1.0,1.0,0.0,0
очки,0.0,0.0,1.0,0
усы,0.0,1.0,0.0,0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Pragmatic listener


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


Здесь так не получится, поскольку при сообщении "шляпа" P(r2) > P(r1)

#### 4. Применение RSA
Ничего супернеобычного мне в голову не пришло, только рассматривать различную вариативность, например в грамматических конструкциях